<a href="https://colab.research.google.com/github/maayancohen/felyx/blob/main/felyx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Ingestion pipeline: PostgreSQL


Create PotgreSQL database


In [70]:
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# avoid printing connections
con_py = "/usr/local/lib/python3.6/dist-packages/sql/connection.py"
!sed -i 's/ print(cls.connection_list())/ if len(cls.connections)>1:print(cls.connection_list())/' $con_py
# setup client with magic

 * Starting PostgreSQL 10 database server
   ...done.
CREATE ROLE


Connect to the database


---





In [71]:
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres


/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


'Connected: @postgres'

Create reservations table

---



In [76]:
%%sql
CREATE TABLE reservations
(
  rowid serial,
  id         integer,
  customer_id numeric,
  start_latitude  numeric(16,6),
  start_longitude numeric(16,6),
  srid      integer,
  net_price numeric(16,6)
)

(psycopg2.ProgrammingError) relation "reservations" already exists

[SQL: CREATE TABLE reservations
(
  rowid serial,
  id         integer,
  customer_id numeric,
  start_latitude  numeric(16,6),
  start_longitude numeric(16,6),
  srid      integer,
  net_price numeric(16,6)
)]
(Background on this error at: http://sqlalche.me/e/13/f405)


In [83]:
# Truncate table for testing purposes (idempotency)
%%sql
TRUNCATE table reservations

""


In [84]:
# Upload reservations csv file
%%sql
COPY reservations(rowid,id,customer_id,start_latitude,start_longitude,srid,net_price)
FROM '/content/assignment_reservations.csv' DELIMITER ',' CSV HEADER;

""


In [86]:
# Check result
%sql SELECT * FROM reservations

,rowid,id,customer_id,start_latitude,start_longitude,srid,net_price
0,0,3190619,21437,51.944996,4.475996,28992,58.000000
1,1,3190621,170043,51.913620,4.451897,28992,194.000000
2,2,3190622,146282,51.936062,4.466082,28992,83.000000
3,3,3190623,186654,51.931694,4.504046,28992,69.000000
4,4,3190624,182408,51.924194,4.468743,28992,142.000000
...,...,...,...,...,...,...,...
9995,9995,3219665,25471,51.917927,4.473958,28992,176.000000
9996,9996,3219666,38605,51.904621,4.486405,28992,103.000000
9997,9997,3219669,187030,51.924500,4.470992,28992,344.000000
9998,9998,3219671,55212,51.919949,4.480230,28992,139.000000


# Cloud Firestore (Firebase): NoSQL

In [2]:
#/content/sodium-inverter-firebase-adminsdk-70ewr-0aa56175fa.json
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore


# Use a service account
cred = credentials.Certificate('/content/sodium-inverter-firebase-adminsdk-70ewr-b175aa415b.json')
default_app = firebase_admin.initialize_app(cred)
db = firestore.client()


# Sample 10 random documents from the reservation collection
doc_ref = db.collection(u'reservations').limit(10).stream()

for doc in doc_ref:
    if doc.exists:
        print(doc.to_dict())
#        print(f'{doc.to_dict()}')
    else:
        print(u'No such document!')
firebase_admin.delete_app(default_app)



{'srid': 28992, 'customer_id': 21437, 'start_longitude': 4.475996, 'net_price': 58, 'id': 3190619, 'start_latitude': 51.944996}
{'start_latitude': 51.91362, 'id': 3190621, 'srid': 28992, 'customer_id': 170043, 'start_longitude': 4.451897, 'net_price': 194}
{'net_price': 462, 'id': 3190640, 'start_latitude': 51.923218, 'srid': 28992, 'customer_id': 56766, 'start_longitude': 4.486422}
{'customer_id': 25270, 'srid': 28992, 'start_longitude': 4.497288, 'net_price': 142, 'id': 3190842, 'start_latitude': 51.928139}
{'start_latitude': 51.915543, 'id': 3193307, 'customer_id': 22578, 'srid': 28992, 'start_longitude': 4.47678, 'net_price': 326}
{'start_latitude': 51.927845, 'id': 3193308, 'srid': 28992, 'customer_id': 17681, 'start_longitude': 4.510067, 'net_price': 377}
{'srid': 28992, 'customer_id': 179847, 'start_longitude': 4.447287, 'net_price': 179, 'start_latitude': 51.915932, 'id': 3193310}
{'net_price': 333, 'id': 3193313, 'start_latitude': 51.929993, 'customer_id': 17681, 'srid': 28992

# Exposing the endpoints as Flask app routes

In [ ]:
!pip install flask-ngrok

In [106]:
from flask import Flask

app = Flask(__name__ ,template_folder="drive/MyDrive/Colab Notebooks/templates")
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route('/')
def home():
  return 'Hello World'

if __name__ == '__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7990b88be97f.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [21/Jan/2021 17:40:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2021 17:40:25] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
#@title Default title text
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template
app = Flask(__name__ ,template_folder="drive/MyDrive/Colab Notebooks/templates")
run_with_ngrok(app)   #starts ngrok when the app is run


@app.route("/")
def home():
    return render_template('index.html', title="endpoint1")

@app.route("/endpoint2")
def docs():
    return render_template("index.html", title="endpoint2")

@app.route("/endpoint3")
def about():
    return render_template("index.html", title="endpoint3")


app.run()

# Reservation Exposed With Cloud Functions:


[Cloud function 1: Reservation Data](https://europe-west1-sodium-inverter.cloudfunctions.net/felyx-reservations)

[Cloud function 1: Reservation Schema Structure](https://europe-west1-sodium-inverter.cloudfunctions.net/felyx-reservations-sql-schema)